In [1]:
# System libraries:
%load_ext autoreload
import pandas as pd
import json
import nltk
import numpy as np
import os
import sys
from IPython.core.interactiveshell import InteractiveShell
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
InteractiveShell.ast_node_interactivity = "all"

In [5]:
yle_fi_local_lemmas = pd.read_csv('data/processed/yle_fi_merged.csv', header=None)

In [6]:
yle_fi_local_lemmas

# Raw tokens,ids, years

,0,1,2,3,4
0,Kaukasian konflikti: Aseissa Georgia ja Venäjä...,Kaukasiassa Georgian ja siitä eroon pyrkivän E...,Kaukasian konflikti: Aseissa Georgia ja Venäjä...,2008-08-11T11:03:41+0300,"['Kaukasia', 'konflikti', ':', 'ase', 'Georg',..."
1,Otteita Venäjän-tuntijoiden vastauksista,Tähän tiivistelmään on koottu otteita asiantun...,Otteita Venäjän-tuntijoiden vastauksista\n\nTä...,2007-01-14T12:46:58+0200,"['ote', 'venäjä', 'vasta', 'tämä', 'tiivistelm..."
2,Etelä-Ossetian ruutitynnyri räjähti viimein,Perjantaina 8. elokuuta 2008 pitkään jännittyn...,Etelä-Ossetian ruutitynnyri räjähti viimein\n\...,2008-08-12T21:33:32+0300,"['Etelä-Ossetia', 'ruuti', 'räjähtää', 'viimei..."
3,Yle Uutiset seuraa Ukrainan kriisiä hetki hetk...,Venäjän duuma käsittelee tänään Krimin ja Seva...,Yle Uutiset seuraa Ukrainan kriisiä hetki hetk...,2014-03-18T09:06:59+0200,"['Yle', 'uutinen', 'seura', 'ukraina', 'kriisi..."
4,Saksan ulkopolitiikan täyskäännös,Tuskin mikään muu Euroopan maa on viime kuukau...,Saksan ulkopolitiikan täyskäännös\n\nTuskin mi...,2014-10-02T12:30:30+0300,"['saksa', 'ulko-', 'täyskäännös', 'tuska', 'mi..."
5,Viennin rasvatahra: käsikirjoitus,"NUMMELA: ""Mä vastaan ihan niinkuin asia on mei...",Viennin rasvatahra: käsikirjoitus\n\nNUMMELA: ...,1998-11-30T00:00:00+0200,"['vienti', 'rasva', ':', 'käsi', 'NUMMELA:', '..."
6,"Näkökulma: ""Venäjä, Venäjä, Venäjä"" – Suomen k...",Seitsemän vuotta sitten puolustusministeri Jyr...,"Näkökulma: ""Venäjä, Venäjä, Venäjä"" – Suomen k...",2014-11-19T08:26:21+0200,"['näkö', ':', '""', 'venäjä', ',', 'venäjä', ',..."
7,Venäjä-Suomi minuutti minuutilta,Suomen jääkiekkomaajoukkue kohtasi MM-kisojen ...,Venäjä-Suomi minuutti minuutilta\n\nSuomen jää...,2010-05-18T20:18:46+0300,"['Venäjä-Suomi', 'minuutti', 'minuutti', 'suom..."
8,Näin Suomi voitti Venäjän rankkareilla,Suomen jääkiekkomaajoukkue voitti välisarjan p...,Näin Suomi voitti Venäjän rankkareilla\n\nSuom...,2011-05-09T16:39:49+0300,"['nämä', 'suomia', 'voittaa', 'venäjä', 'rankk..."
9,Jääkiekon MM-finaali minuutti minuutilta,Jääkiekon MM-kisat huipentuivat sunnuntai-illa...,Jääkiekon MM-finaali minuutti minuutilta\n\nJä...,2010-05-23T20:41:34+0300,"['jää', 'MM', 'minuutti', 'minuutti', 'jää', '..."
